# 划分成10个fold，且同一艺术家的歌必须放在不同fold里（Music Delta除外）
## 结果存在train_validation_test_split.txt里，除非重新划分，否则这个文件不必再运行了。

### 问题：随机的策略不够好，有时候可能找不到合适的划分方式，就死循环了

In [14]:
import medleydb as mdb
import random

In [16]:
artist_title_dict = {}
artist_set = set()
multitrack_generator = mdb.load_all_multitracks()
for multitrack in multitrack_generator:
    if multitrack.has_melody:
        if not multitrack.artist in artist_title_dict.keys():
            artist_title_dict[multitrack.artist] = []
        artist_title_dict[multitrack.artist].append(multitrack.title)
        artist_set.add(multitrack.artist)
        
artist_title_dict['AimeeNorwich'].remove('Child')
artist_set.remove('MusicDelta')
artist_set = list(artist_set)

fold = 10

min_num_per_fold = 107//10
max_num_per_fold = min_num_per_fold + 1
max_fold_num = 107%min_num_per_fold
min_fold_num = fold - max_fold_num

fold_num_list = [min_num_per_fold] * min_fold_num + [max_num_per_fold] * max_fold_num

global_result = []
flag = False
MusicDelta_list = artist_title_dict['MusicDelta']

for fold_index in range(fold):
    result = []
    if not flag:
        while len(result) < fold_num_list[fold_index]:
            random_artist_index = random.randint(0, len(artist_set)-1)
            while len(result) + len(artist_title_dict[artist_set[random_artist_index]]) > fold_num_list[fold_index]:
                random_artist_index = random.randint(0, len(artist_set)-1)
            # OK, find an adequate artist
            artist = artist_set[random_artist_index]
            artist_set.remove(artist)
            for title in artist_title_dict[artist]:
                result.append('_'.join((artist, title)))
            if len(artist_set) == 0:
                flag = True
                len_left = fold_num_list[fold_index]-len(result)
                result += ['MusicDelta_' + i for i in MusicDelta_list[-len_left:]]
                MusicDelta_list = MusicDelta_list[:-len_left]
                break
    else:
        result += ['MusicDelta_' + i for i in MusicDelta_list[-fold_num_list[fold_index]:]]
        MusicDelta_list = MusicDelta_list[:-fold_num_list[fold_index]]
    global_result.append(result)

In [23]:
with open('./train_validation_test_split.txt','w') as f:
    for l in global_result:
        for i in l:
            f.write(i+'\n')
        f.write('\n')